In [2]:
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

In [61]:
cases = pd.read_csv("conposcovidloc.csv")
cases = cases[(cases['Accurate_Episode_Date'] >= '2020-07-01') & (cases['Accurate_Episode_Date'] < '2020-11-01')]
cases.head()

,Row_ID,Accurate_Episode_Date,Case_Reported_Date,Test_Reported_Date,Specimen_Date,Age_Group,Client_Gender,Case_AcquisitionInfo,Outcome1,Outbreak_Related,Reporting_PHU_ID,Reporting_PHU,Reporting_PHU_Address,Reporting_PHU_City,Reporting_PHU_Postal_Code,Reporting_PHU_Website,Reporting_PHU_Latitude,Reporting_PHU_Longitude
90,91,2020-07-02,2020-07-03,2020-07-03,2020-07-02,30s,MALE,CC,Resolved,NaN,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893
162,163,2020-07-10,2020-07-11,2020-07-11,2020-07-10,50s,MALE,CC,Resolved,Yes,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893
202,203,2020-07-07,2020-07-09,2020-07-09,2020-07-07,40s,FEMALE,CC,Resolved,NaN,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893
214,215,2020-07-01,2020-07-07,2020-07-07,2020-07-06,40s,MALE,OB,Resolved,NaN,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893
216,217,2020-07-10,2020-07-11,2020-07-11,2020-07-10,50s,FEMALE,NO KNOWN EPI LINK,Resolved,NaN,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893


### Create the dimension for Patient

#### Subset the columns for Patient 

In [92]:
Patient_dimension = cases[['Row_ID','Client_Gender','Age_Group','Case_AcquisitionInfo', 'Outbreak_Related']].copy()
Patient_dimension.columns = ['Patient_ID', 'Gender', 'Age_Group', 'Acquisition_Group', 'Outbreak_Related']

#### Make a community spread class in Acquisition Group

In [93]:
 # A ‘case with an epidemiological link’ is a case that has either been exposed to a confirmed case, 
 # or has had the same exposure as a confirmed case (e.g. eaten the same food, stayed in the same hotel, etc).
 # https://deputyprimeminister.gov.mt/en/health-promotion/idpcu/Pages/casedefinition.aspx
Patient_dimension['Acquisition_Group'].replace({'NO KNOWN EPI LINK': 'CS', 'UNSPECIFIED EPI LINK':'CS'}, inplace = True)
Patient_dimension.Acquisition_Group.unique()

array(['CC', 'OB', 'CS', 'TRAVEL', 'MISSING INFORMATION'], dtype=object)

#### Make Outbreak_Related column boolean and remove NaN values

In [94]:
Patient_dimension['Outbreak_Related'].replace({'Yes': '1'}, inplace = True)
Patient_dimension.Outbreak_Related.fillna(0, inplace = True)

In [97]:
Patient_dimension.insert(0, "Patient_surrogate_key",  np.arange(len(Patient_dimension)))

In [98]:
Patient_dimension.head()

,Patient_surrogate_key,Patient_ID,Gender,Age_Group,Acquisition_Group,Outbreak_Related
90,0,91,MALE,30s,CC,0
162,1,163,MALE,50s,CC,1
202,2,203,FEMALE,40s,CC,0
214,3,215,MALE,40s,OB,0
216,4,217,FEMALE,50s,CS,0
